# DyNetworkX Tutorial – Using Predictive Component for Compound Slices

The objective of this tutorial is to showcase a typical use case of DyNetworkX's predictive component for compound slices.
We will be using a Facebook Wall Post dataset, a preview of the data file can be seen below.
Facebook Wall Post dataset will be downloaded to current directory if not already present. (Size: 19 MB)

In [7]:
import os
import urllib

file_path = "facebook-wall.txt.anon"
if not os.path.exists(file_path):
    import gzip

    print("Downloading Facebook wall-posts dataset from http://socialnetworks.mpi-sws.mpg.de/data/facebook-wall.txt.gz ...")
    urllib.request.urlretrieve("http://socialnetworks.mpi-sws.mpg.de/data/facebook-wall.txt.gz", file_path + '.gz')

    with gzip.open(file_path + '.gz', 'rb') as zip_file:
        with open(file_path, "wb") as file:
            file.write(zip_file.read())

    os.remove(file_path + '.gz')
    print("Download complete.")

with open(file_path, "r") as file:
    for x in range(5):
        print(next(file))

28	28	1095135831

1015	1017	1097725406

959	959	1098387569

991	991	1098425204

1015	1017	1098489762



### Loading Data into DyNetworkX

Loading data a text file using the function `dnx.ImpulseGraph.load_from_txt`.
Make sure to specify necessary arguments such as `delimiter`, `timestamptype`, and `order`.

Comparing the output of the new ImpulseGraph, it is possible to verify the data set is correctly imported. (Note: order not guaranteed)

*It should be noted, the `load_from_txt` function allows the optional parameter `predict` to automate the predictive process,
however for the sake of example, we will keep the default value of False to illustrate how to preform this process manually.*

In [8]:
import dynetworkx as dnx

impulseG = dnx.ImpulseGraph.load_from_txt("facebook-wall.txt.anon", delimiter="\t", timestamptype=int, order=('u', 'v', 't'), predict=False)

print(impulseG.edges()[:5])

[(3186, 6564, 1207197810), (12054, 12082, 1226542537), (20736, 20338, 1184687065), (30828, 43401, 1224900141), (16327, 11065, 1193210633)]


### Generating Predictive Model

While it is possible to use IntervalGraph, ImpulseGraph, and their extensions
without the use of the predictive model, for analysis involving repeated use of compound slices, users may see computation time performance by doing so.
Thankfully, creating a predictive model is as easy as one line!

Users should call this line after all edges of interest have been added to the graph.
`generate_predictive_model` accepts an optional integer value indicating the number of training samples for the predictive model.

In [9]:
impulseG.generate_predictive_model(250)

### Continue As Normal

From this point forward, continue using the DyNetworkX structure as normal.
When a compound slice is requested while a model has been trained, a prediction is made and the faster sub-structure is chosen.
When no model is trained, all compound slices are performed node-first.

For more information about the predictive compound component of DyNetworkX: https://arxiv.org/abs/2206.11444

### Node Slice

A node slice is any slice that places constraints on the requested nodes, but not the interval.

In [10]:
print(impulseG.edges(u=[1939, 6748])[:5])

[(1939, 11698, 1218747420),
 (1939, 5426, 1224555311),
 (15420, 1939, 1221978053),
 (1578, 1939, 1222055866),
 (19753, 1939, 1230770835)]

### Interval Slice

An interval slice is any slice that places constraints on the requested interval, but not the nodes.

In [11]:
print(impulseG.edges(begin=1185999602, end=1285999602)[:5])

[(13162, 6748, 1185999602),
 (20339, 25081, 1185999692),
 (9822, 1795, 1185999721),
 (8758, 11733, 1185999744),
 (20339, 25081, 1185999745)]

### Compound Slice

A compound slice is any slice that places contraints on both the requested nodes and interval.

In [12]:
print(impulseG.edges(u=[1939, 6748], begin=1185999602, end=1285999602)[:5])

[(1939, 11698, 1218747420),
 (6748, 3666, 1226858138),
 (1578, 1939, 1222055866),
 (6748, 10929, 1186340031),
 (13162, 6748, 1188944828)]